# CRISPR_DeepEnsemble


In [ ]:
# Import and set defaults
import CRISPR_DeepEnsemble
import torch as t
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr 

t.manual_seed(123)
dtype = t.float64
t.set_default_dtype(dtype)

## Data

Here we have provided some sample data in the "data" directory. \
Ensure the data is formatted as follows:




In [ ]:
# Data selection
workingDir = "data/"
# Samples (One hot encoded 30mer)
Samples = "merged_X.pt"
# Features (30mer melting point)
Features = "merged_Features.pt"
# Response (30mer Indel frequency)
Response = "merged_Y.pt"

In [ ]:
# Example Implementation 

# Load data
S = t.load(f"{workingDir}/{Samples}").transpose(1,2).type(dtype)
F = t.load(f"{workingDir}/{Features}").reshape(-1,1).type(dtype)
# Scale response to ensure it is (0,1)
y = t.clamp((t.load(f"{workingDir}/{Response}")/100).type(dtype), 1e-3, 0.999)

# Shuffle the dataset 
perm = t.randperm(len(y)) 
S, F, y = S[perm, :, :], F[perm,:], y[perm]  

# Divide into test and train 
test_sample_count = 20000

S_test = S[-test_sample_count:,:,:].unsqueeze(dim=1)  
F_test = F[-test_sample_count:,:]
y_test =  y[-test_sample_count:]

S_train = S[:-test_sample_count,:,:].unsqueeze(dim=1)  
F_train = F[:-test_sample_count,:]
y_train = y[:-test_sample_count]

# Data loader used to train model
myData = CRISPR_DeepEnsemble.Seqs_and_Features(S=S[:-test_sample_count,:,:], F=F[:-test_sample_count,:], y=y[:-test_sample_count]) 

In [ ]:
# Training a model, simply create a new RegressionDeepEnsemble object and call its `train_ensemble` method.
myEnsemble = CRISPR_DeepEnsemble.RegressionDeepEnsemble(BaseNet=CRISPR_DeepEnsemble.CRISPRnet, dataset=myData,
                                    n_estimators=25, batch_size=1000,
                                    response_var = t.distributions.Beta)
myEnsemble.train_ensemble(n_epochs=5)

In [ ]:
# Example uncertainty plot
L,U,IQR = myEnsemble.uncertainty_bounds(inputs =(S_test, F_test), n_samples=1000,
                                    lower=0.01, upper=0.99) 
pred = myEnsemble.predict(inputs = (S_test, F_test))

# below line gives sorting order for some input (e.g., prediction, true value, or uncertainty bounds)
# _ids = t.argsort(y_test, descending=True)
# ids = _ids[:30]

# below line show the first 30 results
ids = range(30)

myEnsemble.plot_uncertainties(inputs=(S_test[ids,:,:,:], F_test[ids,:]), 
                              true_vals = y_test[ids], plot_means=False,
                              lower=0.025, upper=0.975)

In [ ]:
# Compute Spearman correlation between predictions and true answers in test set
vals = (pred.numpy(), y_test)
print(f"Spearman: {round(spearmanr(*vals)[0],3)}, Pearson: {round(pearsonr(*vals)[0],3)}")